In [1]:
# LLM Agents in Google ADK: Packing List & Itinerary Example
# Welcome 👋🏻 In this notebook, you'll install Google’s Agent Development Kit (ADK), 
# configure your environment, define two LLM agents, and run your agent interactions—all in just a few steps.
# You'll see how LLM Agents power real-world travel apps like WanderWise, using natural language understanding, 
# reasoning, and tool integration to deliver smart, actionable results.

In [2]:
# load API Key set in .EnvironmentError
import dotenv
dotenv.load_dotenv()

import os
#import getpass

# Load the Gemini API key from environment variable
# If not set, prompt the user to enter it
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    print("GEMINI_API_KEY not found in environment variables.")
    #api_key = getpass.getpass("Enter your GEMINI_API_KEY: ")
    #os.environ["GEMINI_API_KEY"] = api_key

print("API key configured successfully." if api_key else "Failed to set API key.")

API key configured successfully.


In [3]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.18.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: c:\Users\gabri\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updat

In [5]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [6]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

# InMemorySessionService: A simple, in-memory service for managing conversation sessions. 
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

from IPython.display import Markdown, display

In [7]:
# Modular Function for Agent Interaction
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    # This flexibility allows for both isolated examples and continuous conversations.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. In a multi-turn conversation, you would reuse
    # the same session_id across turns to maintain context. 
    # For these distinct examples, we'll use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object, which is how messages
    # are structured for the agent.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner: This is the engine that connects the user's message
    # to the agent and manages the session state.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent: This sends the user's message to the agent for processing.
    # The 'runner.run()' method returns an iterable of events that describe
    # the agent's actions and responses.
    events = list(runner.run(user_id=user_id, session_id=session_id, new_message=content))
    # Copilot added the list(), before it was:
    #  events = runner.run(user_id=user_id, session_id=session_id, new_message=content)
    # See Copilot fix notes at the bottom for more info
    
    # ** Suggested by Copilot to print the raw event and each part to inspect available fields **
    # Print the raw event and each part to inspect available fields:
    if False:  # Set to True to enable debug printing
        for event in events:
            print("EVENT:", event)
            for i, part in enumerate(event.content.parts):
                print(" PART", i, "repr:", repr(part))
                # Show common possible attributes
                for attr in ("text", "html", "url", "uri", "href", "link"):
                    if hasattr(part, attr):
                        print("  ", attr, "=", getattr(part, attr))

    # This is the new version created by Copilot to handle the links in grounding metadata
    # before it was:
    #     # Process the events to find and extract the final response from the agent.      
    #     for event in events:
    #         if event.is_final_response():
    #             # Concatenate all parts in the final response
    #             response = ''.join(part.text for part in event.content.parts if part.text)
    #             return response

    # Helper to get readable text for a Part
    def _part_to_text(part):
        if getattr(part, "text", None):
            return part.text
        if getattr(part, "html", None):
            return part.html
        for attr in ("url", "uri", "href", "link"):
            val = getattr(part, attr, None)
            if val:
                return f"[{val}]({val})"
        return str(part)

    # Build final response and include grounding links if available
    for event in events:
        if event.is_final_response():
            # Combine the parts' text/HTML
            response = "".join(_part_to_text(p) for p in event.content.parts)

            # Try to append grounding sources (if present)
            sources = []
            gm = getattr(event, "grounding_metadata", None)
            if gm and getattr(gm, "grounding_chunks", None):
                for chunk in gm.grounding_chunks:
                    web = getattr(chunk, "web", None)
                    if web:
                        title = getattr(web, "title", None) or getattr(web, "uri", None)
                        uri = getattr(web, "uri", None)
                        if uri:
                            # Use the title if present, otherwise display the URL as the label
                            label = title if title else uri
                            sources.append(f"- [{label}]({uri})")

            if sources:
                response += "\n\n**Sources:**\n" + "\n".join(sources)

            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

# # NOTE: More details about Runners, Sessions, and advanced event processing are coming up in later tutorials—stay tuned!

Modular function `run_adk_agent_interaction` successfully defined.


In [8]:
# Define the Packing List Agent
# Our first agent, the packing_list_agent, is designed to generate a comprehensive packing list based on a user's travel details. 
# The key here is its instruction set, which guides the agent to ask clarifying questions and organize its output in a clear, categorized Markdown format.
# name: A unique identifier for the agent.
# model: The specific Gemini model (gemini-2.0-flash) powering the agent's reasoning.
# description: A brief summary of the agent's function.
# instruction: The detailed prompt that defines the agent's behavior, including what information to seek and how to format the output.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that generates a packing list for a trip given a destination, duration, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        When the user provides a destination and trip duration, generate a detailed packing list tailored for that location and timeframe.
        Ask about planned activities and any special preferences or needs (such as business, hiking, or traveling with children).
        If the user wishes, you may ask about personal preferences (such as specific clothing or toiletries), but do not assume based on gender.
        Make sure your suggestions are practical and organized by category (clothing, toiletries, electronics, documents, etc.).
        If the user provides additional details, refine the list accordingly.
        Present the packing list in a clear and easy-to-read markdown format.
    """,
)
# NOTE: In future lessons, you will integrate external tools (e.g., weather APIs) for even more personalized lists.
print("`packing_list_agent` successfully defined.")

`packing_list_agent` successfully defined.


In [9]:
#  Define common parameters for our interactions
APP_NAME = "wanderwise_app"
USER_ID = "user_001"

# Create a common session service instance to reuse.
common_session_service = InMemorySessionService()

# Define the user's input for the packing list agent.
PACKING_LIST_USER_INPUT = "I’m a 21 yo female traveling to Banff for 5 days in July. I plan to do some easy hiking and some city sightseeing."
PACKING_LIST_SESSION_ID = f"{APP_NAME}_packing_list_session_001" # Unique session ID for this interaction

print("--- Running Packing List Agent ---")

# Invoke the packing list agent using our modular helper function.
packing_list_response = await run_adk_agent_interaction(
    agent=packing_list_agent,
    user_id=USER_ID,
    session_id=PACKING_LIST_SESSION_ID,
    input_text=PACKING_LIST_USER_INPUT,
    app_name=APP_NAME,
    session_service=common_session_service
)

# Display the agent's response, rendering the Markdown for a nicely formatted list.
display(Markdown(f"{packing_list_response}"))

print("--- Packing List Agent Interaction Complete ---")

--- Running Packing List Agent ---


Okay, I can help you create a packing list for your 5-day trip to Banff in July! Since you'll be doing some easy hiking and sightseeing, I'll keep that in mind.

Here's a preliminary packing list, which I can refine further based on your preferences:

**Clothing:**

*   **Tops:**
    *   5-7 Short-sleeved shirts (moisture-wicking material recommended for hiking)
    *   1-2 Long-sleeved shirts (for layering and sun protection)
    *   1 Light fleece or sweater
*   **Bottoms:**
    *   1-2 pairs of hiking pants or leggings
    *   1 pair of jeans or casual pants
    *   1 pair of shorts or a skirt (weather dependent)
*   **Underwear:** 5-7 pairs
*   **Socks:**
    *   5-7 pairs of everyday socks
    *   2-3 pairs of hiking socks (wool or synthetic)
*   **Outerwear:**
    *   1 Light jacket (water-resistant is a plus)
*   **Sleepwear:** 1-2 sets
*   **Swimsuit:** (if you plan on visiting any hot springs or pools)

**Shoes:**

*   Hiking shoes or sturdy walking shoes
*   Casual shoes or sandals

**Toiletries:**

*   Sunscreen (high SPF)
*   Insect repellent
*   Shampoo, conditioner, and body wash
*   Toothbrush, toothpaste, and floss
*   Deodorant
*   Face wash and moisturizer
*   Any personal hygiene items

**Electronics:**

*   Phone and charger
*   Camera and charger
*   Portable power bank (optional)

**Documents:**

*   Driver's license or ID
*   Any relevant reservations or tickets

**Other:**

*   Backpack for day hikes
*   Water bottle
*   Sunglasses
*   Hat
*   Small first-aid kit
*   Lip balm with SPF

To make this list even better, could you tell me:

*   **Are there any specific brands or types of clothing you prefer?**
*   **Do you have any favorite toiletries you can't live without?**
*   **Are there any specific activities you are interested in?**
*   **Do you have any allergies or medical conditions I should consider when suggesting items?**

Once I have this information, I can customize the list to perfectly suit your needs!


--- Packing List Agent Interaction Complete ---


In [10]:
# Congratulations 🎉  You’ve just defined and successfully run your first LLM agent for a real-world task: generating a personalized packing list. 
# This demonstrates how easily you can instruct an LLM to perform structured tasks and manage user interactions within the ADK framework.

In [11]:
# Itinerary Agent with Google Search Tool
# Now, let's go a step further: create an Itinerary Agent that uses the google_search tool! 
# This agent will be capable of generating a personalized itinerary for your trip, 
# leveraging live search results to find up-to-date recommendations for attractions, restaurants, and activities.

In [12]:
# Import the Google Search tool from ADK.
# This tool provides the agent with the ability to perform web searches.
from google.adk.tools import google_search

print("`google_search` tool imported.")

`google_search` tool imported.


In [13]:
# Define the Itinerary Agent
# The agent's instruction should explicitly guide it on when and how to use these tools. 
# Notice the instruction use the [google_search] tool. 
# This syntax helps the LLM understand that it has access to a tool named google_search and encourages it to use it appropriately.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, include it as a Markdown hyperlink like [Item Name](URL).
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
        Don't forget to include the links in the answer as a Markdown hyperlink like [Item Name](URL).
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

`itinerary_agent` successfully defined with `google_search` tool.


In [14]:
# Run the Itinerary Agent
ITINERARY_USER_INPUT = "I'm visiting Barcelona for 3 days and I love art, food, and dance. Can you plan my trip?"
ITINERARY_SESSION_ID = f"{APP_NAME}_itinerary_session_001" # Unique session ID for this interaction

print("--- Running Itinerary Agent with Google Search ---")

# Invoke the itinerary agent using our modular helper function.
# This agent will internally decide to use the `google_search` tool based on its instructions.
itinerary_response = await run_adk_agent_interaction(
    agent=itinerary_agent,
    user_id=USER_ID,
    session_id=ITINERARY_SESSION_ID,
    input_text=ITINERARY_USER_INPUT,
    app_name=APP_NAME,
    session_service=common_session_service
)

# Display the agent's response, rendering the Markdown.
display(Markdown(f"{itinerary_response}"))

print("--- Itinerary Agent Interaction Complete ---")

--- Running Itinerary Agent with Google Search ---


Okay, I'd love to help you plan your 3-day trip to Barcelona! I will focus on art, food, and dance as you requested. Let's get started!

Here is a possible itinerary for your 3-day trip to Barcelona:

### Day 1

*   **Morning:** Start your day by visiting the [Sagrada Familia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFfe-VS96XlAZKdgbn73jqTkQ3CiU9uUpm9jhje4v3mvqUzaGR7kN57Vn5XBwJ2IbhxxfH52jy3DjTQYDhQBg9WypT0ioU0LdDAJjvXkr0vviNswYH1nGKVlfPGGyakN8R7y_4Hsi1lOGAiBMIgYV5bMNynoHiojw-gdtQ=). This is a must-see attraction in Barcelona, and it's recommended to book your tickets in advance.
*   **Lunch:** Have lunch at [El Quim de la Boqueria](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLe02bJMjR4cbIf8DCzg_sZuRuhXBOg0FSvGN5_G1xncjNx8v0jWJJRpuvcqOWgGkrETVPGd4zKcwwXKpWd10aMRF47DLs7OGQiCPPFQ9jpWUZ9-sK79EAa8__WrRPOBXK2_v_oQ==), a popular spot in the [Boqueria Market](https://www.boqueria.barcelona/) for Catalan cuisine.
*   **Afternoon:** Explore the [Gothic Quarter](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGEdb68hbl8xxMWiaiU1M79mSJ6YI_QEJHxq1yXQ3lsIu4Ojgn8CXzggZNmK_jZTHKIzRB8t000ORCb2fLYrkLuZZfmQTadKjh4iDtWVsE-W4LRNSizqZP4b2R9Bc-YSaewcTdZ2GUiFGAANHLT2WYRUWLv-pDllcbqgnD67gS3), the oldest part of the city, with its small alleyways, hidden plazas, and historic buildings.
*   **Evening:** Enjoy a [flamenco show at Tablao Flamenco Cordobes](https://www.tablaocordobes.com/en/). You can choose a ticket with a drink or add a buffet dinner.

### Day 2

*   **Morning:** Visit [Park Güell](https://www.parkguell.barcelona/en/), another UNESCO World Heritage Site designed by Antoni Gaudí. Book your tickets in advance.
*   **Lunch:** Have lunch at [Ciutat Comtal](https://maps.app.goo.gl/fEqEJU28UMK9Gq4c6) for some tapas.
*   **Afternoon:** Immerse yourself in art at the [Museu Picasso](https://www.museupicasso.bcn.cat/), which showcases Picasso's early works.
*   **Evening:** Experience the [Magic Fountain of Montjuïc](https://www.barcelona.cat/en/what-to-do/magic-fountain-of-montjuic), with its light, music, and water show.

### Day 3

*   **Morning:** Explore [Casa Batlló](https://www.casabatllo.es/en/) and [Casa Milà (La Pedrera)](https://www.lapedrera.com/en/home), two more of Gaudí's architectural masterpieces.
*   **Lunch:** Enjoy a seafood lunch at [Enoteca Paco Pérez](https://www.hotelartsbarcelona.com/en/enoteca-paco-perez-restaurant-in-barcelona/), a Michelin-starred restaurant with Mediterranean-inspired dishes.
*   **Afternoon:** Visit the [Fundació Joan Miró](https://www.fmirobcn.org/en/), which offers an overview of Miró's surrealist world.
*   **Evening:** Consider a [sailing trip along Barcelona's coastline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGtLjkOnU7rLMb5zROJXerdhzRd_yJA663TZu32ijSWAJt1U0_r2MqSCuEekCKVTzx9PFBP0ep7O3FscN0ZzeHr9hfdtRBGzmyzZZn6BorNCQ2KuRBcKbBeaOUq6aK0oWH_ZtGd85iqt06rNj4xfg7G8PKkvU0bBQ==) for unique views of the city. Alternatively, check out a [dance performance or show](https://www.danceplace.com/dance-events/city/Barcelona,%20Spain).


**Sources:**
- [earthtrekkers.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFfe-VS96XlAZKdgbn73jqTkQ3CiU9uUpm9jhje4v3mvqUzaGR7kN57Vn5XBwJ2IbhxxfH52jy3DjTQYDhQBg9WypT0ioU0LdDAJjvXkr0vviNswYH1nGKVlfPGGyakN8R7y_4Hsi1lOGAiBMIgYV5bMNynoHiojw-gdtQ=)
- [ticketshop.barcelona](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF5PIkfixw5-JnI-K6OVWhukV7zI0KtcL7QHhCH-GVFe0WjESgW5hwV1EuZJmVEl3AGF_x-oFQRPIXxtjxANKytGQzLEPe-fUNKUeSAq8ER7tvosDtS3PvI)
- [youtube.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEBxiKgi0X8FrML219pfXi8JRfAmptsM4R9KMrrLJQ9ib6WKOiualSVFQf-CqmrxiKHMuwRryaiP3jo8qNLABJn8gAAe29laFGwXNZ2SdCRKGqcDpORt-O7x_0Wgn5neJrv5EHbWck=)
- [barcelona-life.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLe02bJMjR4cbIf8DCzg_sZuRuhXBOg0FSvGN5_G1xncjNx8v0jWJJRpuvcqOWgGkrETVPGd4zKcwwXKpWd10aMRF47DLs7OGQiCPPFQ9jpWUZ9-sK79EAa8__WrRPOBXK2_v_oQ==)
- [lonelyplanet.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGEdb68hbl8xxMWiaiU1M79mSJ6YI_QEJHxq1yXQ3lsIu4Ojgn8CXzggZNmK_jZTHKIzRB8t000ORCb2fLYrkLuZZfmQTadKjh4iDtWVsE-W4LRNSizqZP4b2R9Bc-YSaewcTdZ2GUiFGAANHLT2WYRUWLv-pDllcbqgnD67gS3)
- [barcelona-life.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNq3Yz_lsq6zyQdbrNJojErrcnOvPMQxbyiCMAJhh2tnrC3QZ_MaYP3JTqwXUjLmAZb2SQQz0ZlHOImdIKXlKcJvAfZJ4zG4BfkKrqnP6zwuFVaKAEjcomxf03QWXNZIV0HBa9XsXJMAv1rPM=)
- [webarcelona.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHZGRIVQDajiQQMiPnTugzrHDIK5pZegjNiyIhnRD-vnhLH-a3e9-EddgZ_IVt9WZT-pnW5FBlmC-gHb4FJiNNb0pyDKGtFpf5VRLGr-VpV1XCa3a-RZbXXwdXdbPr5ZMUXvodEAbb7xRBE5mq526ot95hcBIsEzooH9Lw=)
- [trvbox.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGtLjkOnU7rLMb5zROJXerdhzRd_yJA663TZu32ijSWAJt1U0_r2MqSCuEekCKVTzx9PFBP0ep7O3FscN0ZzeHr9hfdtRBGzmyzZZn6BorNCQ2KuRBcKbBeaOUq6aK0oWH_ZtGd85iqt06rNj4xfg7G8PKkvU0bBQ==)
- [getyourguide.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHdvb0UpvXeWX5BwofsvsP8ZIeTFFNe1BV6MSi0-iWk13mhcKJPUsTddnJp1wQbUrlJACXtoGj-AyG34Wf2Cz6HeZlXwgsxpMElerh16gcumfx0bvy5CqGFbfLmFVgEecUa19u27c_xsQbvbp1diYvpKw==)
- [cultureactivities.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFADVx8B2PYBODao-_lTEoPxYCOYvsGDqCKOxh5kNxuavB059fEF0OH62DxQex7sUyvnPGdSEH22TgWOybObWCTjAnPFVF52JrwsgxWLePp_k7uI2MOJCayFpNAIWQ_FFrVjeagrG--u1JzOZz3EgMxHnBwA-KQlXthvkGwn4sn5wFQhxy7HeqqcBV7tmhbcX7xnA==)
- [apartmentbarcelona.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGzNu2C_zlGaxDUX11-wyF2stmC-QC_MFFHdNAM1TBgo_dgXJovfub0jFNuX2w3ebmYXnRtwU_cnlM0KZNrn1DQzld_v6yxWySz3hFA46BAgJuwhDMwLg3Z2eNyhwKY5xdpmaiujoSYJt06OVJhi1UrtCxYYAMQqE_arsSWBQM=)
- [bcn.travel](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFDg5MV9hu_D3Kz5N0ySlRgRepbjwQHJlV0-oQAshzNtsWUzYgQkWc3dewAxw_8BcT80hiqpJ2XYhMnDWmb49zQI2hgbX3MoAD_FojRjUy1WRLpkmSr17N7tspOHec_cDiM42hPf1qbkg==)
- [timeout.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHespy2m0AIEbbNO9IdxqCBkWkX51b4n34ZTeaYMfTYoQge5iNLUgNbSiH3NkBA2UZEEcOy8Uoa9RvUXwZcNRSMYrAwe6MSe1Fqqzc2o1CIK8cVOp9mxPIZOqvlbxnS0n7IaOVHNiSHcGo9erPDqaCTyA9oYzWIo4u0J-jZzor4tHZRC1RUDA==)
- [emmaplunkett.art](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE1j8vSt7nLono5xN2aLiJps4A9R-J6je3Hi2Xx2vaDOd8u5NN1GL0zCXdzp_DQORAerwx4y4kEKTJ8vO7DBbvNWwOdz9gZZfSfWlwJO6yMPq_-kDJEyQgmxuZHBX51kGUabRZwe7Zwsi0f9uRw-3y93e78L90e1B3AyshhcQ==)
- [danceplace.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEux4z6vvyJkhqLDnxCx1l8_9CNECuq0MXPSAgm_nEjtoWQzmgh5OzscWdYrcqzMAbncL0dRoZC42G8Q7JciJdGZmz72MIQ6nsozyvGJuwweWioPhunkwaIlCmlD4gKCpwb_8UrQMMyc2BBMwItL7oKKzm3nsVR)
- [feverup.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFFWwPNsGLOKKlStTlcpV7i3M39ql9Xg4IYv95_hjWxZIm-ggMQ4XNPF9cbJDoMoImpahE6bdIEi0VTQiC9lM9OAeLe0tXFgdpplgLBaaE3inttoKJej7b00xr8st614g_zG5PXFiVP)

--- Itinerary Agent Interaction Complete ---


When I run this exercise the final answer did not include the links, I tried to fix it with copilot (see notes below), but because it did not work, I fixed manually by adding this to the itinerary_agent instructions
"Don't forget to include the links in the answer as a Markdown hyperlink like ```[Item Name](URL)```."


Notes from copilot fix. The original goal was to ensure links were included in the response as in the video, but the Copilot fix, showed the links in the metadata which was not the saem.

Copilot answer

Links are missing because the agent's human-readable text is in event.content.parts[*].text, while the search tool's URLs are stored in the event's grounding metadata (e.g., event.grounding_metadata.grounding_chunks[*].web.uri). Your current code concatenates only part.text, so it never includes those URIs.
Why that happens (from your debug output)

The debug shows parts with only text (the itinerary markdown).
The grounding sources (titles + URIs) appear under grounding_metadata.grounding_chunks[].web.uri.
Since run_adk_agent_interaction only gathers part.text, it omits the grounding chunk URIs entirely.
What to change (safe, robust fix)

Convert the runner events to a list (so we can inspect them and iterate multiple times).
Build the response from parts (keep existing text).
Extract grounding chunks (title + uri) and append a “Sources” Markdown section with links formatted as [title](uri).
Drop-in code you can paste into the run_adk_agent_interaction function

Replace the existing event handling/response assembly with the block below. It handles text, html, common URL attributes (in parts), and the grounding metadata URIs:

```
# Run the agent and collect events into a list (so we can inspect them safely)
events = list(runner.run(user_id=user_id, session_id=session_id, new_message=content))

# (Optional) debug print of events/parts (keep or remove as needed)
for event in events:
    print("EVENT:", event)
    for i, part in enumerate(event.content.parts):
        print(" PART", i, "repr:", repr(part))
        for attr in ("text", "html", "url", "uri", "href", "link"):
            if hasattr(part, attr):
                print("  ", attr, "=", getattr(part, attr))

# Helper to get readable text for a Part
def _part_to_text(part):
    if getattr(part, "text", None):
        return part.text
    if getattr(part, "html", None):
        return part.html
    for attr in ("url", "uri", "href", "link"):
        val = getattr(part, attr, None)
        if val:
            return f"[{val}]({val})"
    return str(part)

# Build final response and include grounding links if available
for event in events:
    if event.is_final_response():
        # Combine the parts' text/HTML
        response = "".join(_part_to_text(p) for p in event.content.parts)

        # Try to append grounding sources (if present)
        sources = []
        gm = getattr(event, "grounding_metadata", None)
        if gm and getattr(gm, "grounding_chunks", None):
            for chunk in gm.grounding_chunks:
                web = getattr(chunk, "web", None)
                if web:
                    title = getattr(web, "title", None) or getattr(web, "uri", None)
                    uri = getattr(web, "uri", None)
                    if uri:
                        # Use the title if present, otherwise display the URL as the label
                        label = title if title else uri
                        sources.append(f"- [{label}]({uri})")

        if sources:
            response += "\n\n**Sources:**\n" + "\n".join(sources)

        return response

return "No final response from agent."
```

How to apply & test

Option A (recommended): I can patch 02_03.ipynb and replace the event-processing code in the modular function with the snippet above, then you re-run the itinerary cell. Tell me to apply the patch and I’ll make the edit.
B (manual): open the cell that defines run_adk_agent_interaction, replace the event-processing block with the snippet, and run the itinerary cell again.
Notes and small caveats

The URIs in your grounding metadata are redirect URLs (Vertex AI grounding redirect links). They will work as links but may not show the final host in the label unless you use the title from the grounding chunk (we use that when available).
If you want the links inserted inline near the exact place they were used in the textual itinerary (rather than as a “Sources” list), we can implement a more advanced step that maps grounding support segments to locations in the text and inserts hyperlinks in-place. That’s slightly more work but doable.
Do you want me to:

Patch 02_03.ipynb now with this change and remove the debug prints, or
Apply the simpler fix and keep debug prints for one more run so you can confirm the exact mapping?

In [15]:
# Congratulations 🎉 You’ve now successfully built and run two different LLM agents in ADK:
# A Packing List Agent for smart, personalized packing suggestions.
# An Itinerary Agent that effectively uses a google_search tool to plan an amazing trip with live, up-to-date recommendations.
# This demonstrates the power of ADK in combining LLM reasoning with external tools to create truly intelligent and practical applications. 
# This is the foundation for building even richer, multi-agent travel assistants and other sophisticated AI solutions with Google ADK.